In [138]:
import pandas as pd

In [ ]:
df=pd.read_csv('combined_data.csv')

In [142]:
df.drop(columns=['Index','date_only'],inplace=True)

In [143]:
df

,datetime,Retweets,Likes,bertweet_score,cardiff_score,vader_score
0,2013-01-01 00:00:00,0.0,0.0,0.113106,0.259274,0.27060
1,2013-01-01 02:24:00,0.0,0.0,0.496756,0.466894,0.69090
2,2013-01-01 04:48:00,0.0,0.0,0.084343,0.222480,0.15960
3,2013-01-01 07:12:00,0.0,0.0,0.502342,0.416070,0.50000
4,2013-01-01 09:36:00,6.0,1.0,0.073976,0.138584,0.22885
...,...,...,...,...,...,...
21225,2018-10-24 12:00:00,0.0,4.0,0.527563,0.508382,0.50000
21226,2018-10-24 14:24:00,0.0,2.0,0.504450,0.473736,0.76335
21227,2018-10-24 16:48:00,0.0,0.0,0.510920,0.473226,0.50000
21228,2018-10-24 19:12:00,2.0,0.0,0.015322,0.282514,0.20030


In [144]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine


sentiment_columns = ['bertweet_score', 'cardiff_score', 'vader_score']
df['mean_sentiment'] = df[sentiment_columns].mean(axis=1)

# Metrics storage
mae = {col: np.mean(np.abs(df[col] - df['mean_sentiment'])) for col in sentiment_columns}
euclidean = {col: np.linalg.norm(df[col] - df['mean_sentiment']) for col in sentiment_columns}
pearson = {col: np.corrcoef(df[col], df['mean_sentiment'])[0, 1] for col in sentiment_columns}
rmse = {col: np.sqrt(np.mean((df[col] - df['mean_sentiment'])**2)) for col in sentiment_columns}
cosine_sim = {col: 1 - cosine(df[col], df['mean_sentiment']) for col in sentiment_columns}

# Metric rankings
metrics = {
    'MAE': sorted(mae, key=mae.get),
    'Euclidean': sorted(euclidean, key=euclidean.get),
    'Pearson': sorted(pearson, key=pearson.get, reverse=True),
    'RMSE': sorted(rmse, key=rmse.get),
    'Cosine Similarity': sorted(cosine_sim, key=cosine_sim.get, reverse=True)
}

# Compute reciprocal ranks
reciprocal_ranks = {model: [] for model in sentiment_columns}

for metric, ranked_models in metrics.items():
    for rank, model in enumerate(ranked_models, 1):
        reciprocal_ranks[model].append(1 / rank)

# Compute Mean Reciprocal Rank (MRR)
mrr_scores = {model: np.mean(ranks) for model, ranks in reciprocal_ranks.items()}

# Display MRR
print("Mean Reciprocal Rank (MRR) for each sentiment model:\n")
for model, score in sorted(mrr_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{model}: {score:.4f}")

# Best model based on MRR
best_mrr_model = max(mrr_scores, key=mrr_scores.get)
print(f"\nBest Overall Model by MRR: {best_mrr_model}")


Mean Reciprocal Rank (MRR) for each sentiment model:

cardiff_score: 1.0000
bertweet_score: 0.5000
vader_score: 0.3333

Best Overall Model by MRR: cardiff_score


In [145]:
best_model=df[['cardiff_score','Likes','Retweets']]

In [146]:
best_model

,cardiff_score,Likes,Retweets
0,0.259274,0.0,0.0
1,0.466894,0.0,0.0
2,0.222480,0.0,0.0
3,0.416070,0.0,0.0
4,0.138584,1.0,6.0
...,...,...,...
21225,0.508382,4.0,0.0
21226,0.473736,2.0,0.0
21227,0.473226,0.0,0.0
21228,0.282514,0.0,2.0


In [ ]:
best_model.to_csv('best_model_Cardiff.csv')